In [88]:
import numpy as np
import pandas as pd
import pickle
from urllib.parse import urlparse
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from imblearn.over_sampling import RandomOverSampler

In [89]:
comments = pd.read_csv('../WikiPages/wikipedia_preprocessed.csv')

In [90]:
x_text = list(comments['comment'])
labels = list(comments['attack'])

labels, uniques = pd.factorize(labels)

In [91]:
comments = pd.DataFrame({'comment': x_text, 'attack': labels})

In [92]:
#drop nulls
comments = comments[comments['comment'].notna()]

#lower case
comments['comment'] = comments['comment'].str.lower()

#remove stop words
comments['comment'] = comments['comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords.words('english')]))

#tokenize
comments['comment'] = comments.apply(lambda row: nltk.word_tokenize(row['comment']), axis=1)

#remove punctiations
comments['comment'] = [list(filter(lambda x: x not in string.punctuation, sentence)) for sentence in comments['comment']]

In [95]:
comments.to_csv("../../data/wiki_data_DLpreprocessed.csv", index=False)

In [96]:
# triple oversample in original paper is similar to equalizing numbers

def oversample(df):
    classes = df.attack.value_counts().to_dict()
    most = max(classes.values())
    classes_list = []
    for key in classes:
        classes_list.append(df[df['attack'] == key])
    classes_sample = []
    for i in range(1,len(classes_list)):
        classes_sample.append(classes_list[i].sample(most, replace=True))
    df_maybe = pd.concat(classes_sample)
    final_df = pd.concat([df_maybe,classes_list[0]], axis=0)
    final_df = final_df.reset_index(drop=True)
    return final_df

comments_oversampled  = oversample(comments)

In [99]:
comments_oversampled.to_csv("../../data/wiki_data_DLpreprocessed_oversampled.csv", index=False)